## 1 模型搭建

In [1]:
# 下载数据
!wget -P /root/jupyter_notebook http://datasrc.tipdm.net:81/python/case/RectalCancer/train_datas.npy
!wget -P /root/jupyter_notebook http://datasrc.tipdm.net:81/python/case/RectalCancer/test_datas.npy
!wget -P /root/jupyter_notebook http://datasrc.tipdm.net:81/python/case/RectalCancer/train_labels.npy
!wget -P /root/jupyter_notebook http://datasrc.tipdm.net:81/python/case/RectalCancer/test_labels.npy

--2022-03-30 11:38:23--  http://datasrc.tipdm.net:81/python/case/RectalCancer/train_datas.npy
正在解析主机 datasrc.tipdm.net... 203.88.218.216
正在连接 datasrc.tipdm.net|203.88.218.216|:81... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度：134960128 (129M)
正在保存至: “/root/jupyter_notebook/train_datas.npy”

100%[======================================>] 134,960,128 91.2M/s   in 1.4s    

2022-03-30 11:38:24 (91.2 MB/s) - 已保存 “/root/jupyter_notebook/train_datas.npy” [134960128/134960128])

--2022-03-30 11:38:25--  http://datasrc.tipdm.net:81/python/case/RectalCancer/test_datas.npy
正在解析主机 datasrc.tipdm.net... 203.88.218.216
正在连接 datasrc.tipdm.net|203.88.218.216|:81... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度：33824128 (32M)
正在保存至: “/root/jupyter_notebook/test_datas.npy”

100%[======================================>] 33,824,128  96.3M/s   in 0.3s    

2022-03-30 11:38:25 (96.3 MB/s) - 已保存 “/root/jupyter_notebook/test_datas.npy” [33824128/33824128])

--2022-03-30 11:38:26--  http://datasrc.tipdm.net:81/python/case/Rect

In [4]:
import numpy as np
import tensorflow as tf

train_datas = np.load('train_datas.npy')
test_datas = np.load('test_datas.npy')
train_labels = np.load('train_labels.npy')
test_labels = np.load('test_labels.npy')

model_path = './tmp/save_model'
num_classes = len(set(train_labels))
data_placeholder = tf.placeholder(tf.float32,[None,200,140,1])
labels_placeholder = tf.placeholder(tf.int32,[None])
dropout_placeholder = tf.placeholder(tf.float32)
def Conv(data,m,n):
    return tf.layers.conv2d(data,m,n,activation=tf.nn.relu)
def Pool(data):
    return tf.layers.max_pooling2d(data,[2,2],[2,2])
conv_1 = Conv(data_placeholder,20,5)
pool_1 = Pool(conv_1)
conv_2 = Conv(pool_1,40,4)
pool_2 = Pool(conv_2)
conv_3 = Conv(pool_2,80,2)
pool_3 = Pool(conv_3)
flatten = tf.layers.flatten(pool_3)
fc = tf.layers.dense(flatten,400,activation=tf.nn.relu)
dropout_fc = tf.layers.dropout(fc,dropout_placeholder)
logits = tf.layers.dense(dropout_fc,num_classes)
predicted_labels = tf.arg_max(logits,1)
losses = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(labels_placeholder,num_classes),
    logits=logits)
mean_loss = tf.reduce_mean(losses)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(losses)

## 2 CNN模型训练

In [5]:
#保存
saver = tf.train.Saver()
def run_train():
    global labels_placeholder
    with tf.Session() as sess:
        print('开始训练模型')
        sess.run(tf.global_variables_initializer())
        batch_size = 60
        input_count = len(train_datas)
        batches_count = int(input_count/batch_size)
        remainder = input_count%batch_size
        print('数据分成%s批，前面每批%s数据，最后一批%s数据'%(batches_count+1,
                                           batch_size,remainder))

        for step in range(1):
            for n in range(batches_count):
                train_feed_dict = {
                    data_placeholder:train_datas[n*batch_size:(n+1)*batch_size],
                    labels_placeholder:train_labels[n*batch_size:(n+1)*batch_size],
                    dropout_placeholder:0.25
                }
                _,mean_loss_val = sess.run([optimizer,mean_loss],
                                           feed_dict = train_feed_dict)
                print('第%s次训练的损失率：%s'%(n,mean_loss_val))
            if remainder >  0:
                start_index = batches_count*batch_size
                train_feed_dict = {
                    data_placeholder:train_datas[start_index:input_count-1],
                    labels_placeholder:train_labels[start_index:input_count-1],
                    dropout_placeholder:0.25
                }
            _,mean_loss_val = sess.run([optimizer,mean_loss],
                                        feed_dict = train_feed_dict)
            print('损失率',mean_loss_val)
            saver.save(sess,model_path)
            print('训练结束，模型保存到：',model_path)

run_train()

开始训练模型
数据分成21批，前面每批60数据，最后一批5数据
第0次训练的损失率：15.353581
第1次训练的损失率：13459.86
第2次训练的损失率：506.6408
第3次训练的损失率：151.99672
第4次训练的损失率：38.60791
第5次训练的损失率：0.9705894
第6次训练的损失率：0.56520534
第7次训练的损失率：0.5657701
第8次训练的损失率：0.69387287
第9次训练的损失率：0.6339985
第10次训练的损失率：0.6666031
第11次训练的损失率：0.60132724
第12次训练的损失率：0.416391
第13次训练的损失率：0.4193592
第14次训练的损失率：0.7718432
第15次训练的损失率：0.66506755
第16次训练的损失率：0.61306155
第17次训练的损失率：0.615803
第18次训练的损失率：0.636338
第19次训练的损失率：0.61064297
损失率 0.577701
训练结束，模型保存到： ./tmp/save_model


## 3 测试模型

In [6]:
def run_test():
    global predicted_labels
    with tf.Session() as sess:
        print('测试模式')
        saver.restore(sess,model_path)
        test_feed_dict = {
            data_placeholder:test_datas,
            labels_placeholder:test_labels,
            dropout_placeholder:0
        }
        predicted_labels_val = sess.run(predicted_labels,
                                        feed_dict = test_feed_dict)
        print(predicted_labels_val)
        return predicted_labels_val

predicted_labels_test = run_test()

accuracy_count = 0
for i in range(len(predicted_labels_test)):
    if predicted_labels_test[i]==test_labels[i]:
        accuracy_count = accuracy_count+1
print('准确率为：',accuracy_count/len(test_labels))

测试模式
INFO:tensorflow:Restoring parameters from ./tmp/save_model
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
准确率为： 0.7450331125827815
